# Filtering the rolled and latent mates 

present in the finger position annotation in the SD302h subset.

In [ ]:
import os
import sys
import shutil
import pandas as pd

sys.path.append(os.path.abspath('../src'))
from utils.file import list_files, make_dirs, get_file_name


## NIST SD302h path

In [ ]:
csv_path = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302h\ebts\latent\finger_positions.csv'

In [ ]:
img_roll_u_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302b\images\baseline\U\500\roll\png'
img_roll_v_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302b\images\baseline\V\500\roll\png'
ebts_latent_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302h\ebts\latent'
latent_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302h\png\latent\lffs\original\1000'


In [ ]:
data = pd.read_csv(csv_path)
display(data.head())
display(data.tail())
data.count()


In [ ]:
data.describe()

## Filter Nan

In [ ]:
data = data.dropna()
display(data.count())
data.describe()


In [ ]:
ct = 0
finger_pos = {'filename':[]}
for d in data.iterrows():
    d = d[1]
    filename = d.filename
    if 'X' in filename:
        continue
    ct += 1
print(ct)
    

## Filter impression

In [ ]:
ct = 0
finger_pos = {'lffs_file':[], 'roll_u_file':[], 'roll_v_file':[]}
for d in data.iterrows():
    d = d[1]
    filename = d.filename
    # print(filename)
    filename_split = filename.split("_")
    subject = filename_split[0]

    # Remove nth impression where n > 1
    if '-2' in filename or '-3' in filename or '-4' in filename:
        continue
    
    roll_fname = '%s_x_%02d.png'%(subject,d.fgp)
    roll_v_fname = roll_fname.replace('x','V_500_roll')
    roll_u_fname = roll_fname.replace('x','U_500_roll')

    if roll_u_fname in finger_pos['roll_u_file']:
        continue
    finger_pos['lffs_file'].append(filename)
    finger_pos['roll_u_file'].append(roll_u_fname)
    finger_pos['roll_v_file'].append(roll_v_fname)
    ct += 1
print(ct)
    

In [ ]:
df_finger_pos = pd.DataFrame(finger_pos)
df_finger_pos

## Filter lffs files, it has mate in roll

In [ ]:
rm_list = []
ct = 0
for d in df_finger_pos.iterrows():
    d = d[1]
    roll_u_fname = d.roll_u_file
    roll_v_fname = d.roll_v_file

    roll_u_path = os.path.join(img_roll_u_dir, roll_u_fname)
    roll_v_path = os.path.join(img_roll_v_dir, roll_v_fname)
    
    if os.path.exists(roll_u_path) and os.path.exists(roll_v_path):
        ct += 1
    else:
        rm_list.append(roll_u_fname)
        print(roll_u_path)
        print(roll_v_path)

print(ct)

for roll_u_fname in rm_list:
    df_finger_pos = df_finger_pos.drop(df_finger_pos[df_finger_pos.roll_u_file == roll_u_fname].index)

In [ ]:
df_finger_pos.to_csv('./NIST_SD302h_mate.csv', index=False)
df_finger_pos

## Copy mate roll and latent image

In [ ]:
db_dir = r'D:\OneDrive\workspace\KSIP\02_Database\00_ORIGINAL_IMAGE'

filtered_latent_dir = os.path.join(db_dir, 'NIST_SD302h_filtered')
filtered_img_roll_u_dir = os.path.join(db_dir, 'NIST_SD302b_filtered/U')
filtered_img_roll_v_dir = os.path.join(db_dir, 'NIST_SD302b_filtered/V')

make_dirs(filtered_latent_dir)
make_dirs(filtered_img_roll_u_dir)
make_dirs(filtered_img_roll_v_dir)

In [ ]:
for d in df_finger_pos.iterrows():
    d = d[1]
    roll_u_fname = d.roll_u_file
    roll_v_fname = d.roll_v_file
    lffs_fname = d.lffs_file
    latent_fname = lffs_fname.replace('.lffs', '.jpg')

    src_path_latent = os.path.join(latent_dir, latent_fname)
    src_path_roll_u = os.path.join(img_roll_u_dir, roll_u_fname)
    src_path_roll_v = os.path.join(img_roll_v_dir, roll_v_fname)

    out_path_latent = os.path.join(filtered_latent_dir, latent_fname)
    out_path_roll_u = os.path.join(filtered_img_roll_u_dir, roll_u_fname)
    out_path_roll_v = os.path.join(filtered_img_roll_v_dir, roll_v_fname)

    shutil.copy(src_path_latent, out_path_latent)
    shutil.copy(src_path_roll_u, out_path_roll_u)
    shutil.copy(src_path_roll_v, out_path_roll_v)
